# Complete Analysis of Carbon Source Variation

© 2019 Griffin Chure. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

--- 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mwc.bayes
import mwc.stats
import mwc.model
import mwc.viz
import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()
colors = mwc.viz.bokeh_theme()
constants = mwc.model.load_constants()

Loading BokehJS ...

In this notebook, we complete analysis of the LacI repressor titration under varying carbon sources. We begin by inferring the calibration factor given all of the data, which we model in a hierarchical fashion. With a repressor count in place, we then estimate the DNA binding energy of each carbon source and compare to that reported in Garcia & Phillips, 2011. 

To begin, we will load the lineage information to compute the calibration factor

## Computing a fluorescence calibration factor 

We begin by loading the data, computing the mean autofluoresnced, and defining the total integrated fluorescence for each cell 

In [40]:
# Load the lineage and fold-change data
lineages = pd.read_csv('../../data/compiled_fluctuations.csv')
fc_data = pd.read_csv('../../data/compiled_fold_change.csv')

# Isolate the autofluorescence data. 
auto_data = fc_data[fc_data['strain']=='auto'].copy()


# Go through and subtract teh autofluorescence information.
for g, d in lineages.groupby(['carbon', 'date', 'run_no']):
    # Extract the mean autofluorescence value for each replicate
    _auto = auto_data[(auto_data['carbon']==g[0]) &(auto_data['date']==g[1]) &
                      (auto_data['run_number']==g[2])].copy()
    lineages.loc[(lineages['carbon'] == g[0]) & (lineages['date']==g[1]) & 
                  (lineages['run_no']==g[2]), 'mean_auto'] = _auto['mean_mCherry'].values.mean()
        
# Subtract the autofluorescence and compute the total intensity.
lineages['I1_tot'] = lineages['area_1'] * (lineages['I_1'] - lineages['mean_auto'])
lineages['I2_tot'] = lineages['area_2'] * (lineages['I_2'] - lineages['mean_auto'])
lineages['summed'] = lineages['I1_tot'] + lineages['I2_tot']
lineages['sq_diff'] = (lineages['I1_tot'] - lineages['I2_tot'])**2
lineages.dropna(inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: Mean of empty slice.
  from ipykernel import kernelapp as app


With this in hand, we can now iterate through each unique day, run_number, and carbon source to infer the calibration factor.  

In [ ]:
model = mwc.bayes.StanModel('../stan/calibration_factor.stan')
stat_dfs = []
for g, d in lineages.groupby(['carbon', 'date', 'run_no']):
    data_dict = {'N':len(d), 'I1':d['I1_tot'], 'I2':d['I2_tot']}
    samples = model.sample(data_dict, iter=3000, control=dict(adapt_delta=0.95))
    stats = model.summarize_parameters()
    stats['carbon'] = g[0]
    stats['date'] = g[1]
    stats['run_no'] = g[2]
    stat_dfs.append(stats)

Found precompiled model. Loading...
finished!
Beginning sampling...
finished sampling!
Beginning sampling...
finished sampling!
Beginning sampling...


In [29]:
# Instantiate the figure. 
carbon = 'glucose'
ax = bokeh.plotting.figure(width=700, height=500,
                           x_axis_type='log',
                           y_axis_type='log',
                           x_axis_label='sum total fluorescence [a.u.]',
                           y_axis_label='squared fluctuations')

c_data = lineages[lineages['carbon']==carbon]

ax.circle(c_data['I1_tot'].values + c_data['I2_tot'].values,
          (c_data['I1_tot'].values - c_data['I2_tot'].values)**2,
         size=1)
        
bokeh.io.show(ax)

In [12]:
data

NameError: name 'data' is not defined

In [37]:
_auto[(_auto['date']==_g[0]) & (_auto['run_number']==_g[1])]['mean_yfp'].values.mean()

164.29079835768732

In [13]:
stat_dfs

[         parameter          mode          mean        median     hpd_min  \
 0            chain      2.000000      2.500000      2.500000           1   
 1        chain_idx    332.000000    750.500000    750.500000           1   
 2           warmup      0.000000      0.000000      0.000000           0   
 3      divergent__      0.000000      0.000000      0.000000           0   
 4         energy__  20527.410971  20532.228826  20531.864469     20527.5   
 5      treedepth__      3.000000      2.902167      3.000000           2   
 6    accept_stat__      0.997820      0.879202      0.945803    0.461756   
 7       stepsize__      0.559582      0.512976      0.517599    0.457125   
 8     n_leapfrog__      7.000000      6.902667      7.000000           3   
 9        tau_alpha      3.491688      2.674400      2.525660  0.00406331   
 10         alpha_1    670.805685    665.390756    664.995185     625.679   
 11  alpha_2_raw[1]      0.433873      0.092018      0.095438    -1.94578   